In [1]:
%load_ext autoreload
%autoreload 2

https://chatgpt.com/share/687d132f-70f0-8003-8800-16c45376f179

i extract subtitle from youtube video
the video is 2hours long
i want summarize with Key points of the video using llm model
so i chunk subtitle every 10 minutes and input each chunk to model to summarize
eg. 1 time summarize for 1 chunk
and then I combine all the sumarize together
I use this method because llm model seem to cunfuse when I input the large amount of data into it
so I chunk it first and do all above
is it a good method or any method suggest that better than this? 

to do
- try chunking in token 
    - 1,000–2,000 tokens per chunk is the safe sweet spot
    - Add 10–20% overlap to catch mid-topic cuts

In [41]:
import requests
from bs4 import BeautifulSoup

url = "https://youtu.be/uDwwwQCeDUc?si=ODvCOaCrGTK9j_ud"

response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
title = soup.title.string.replace(" - YouTube", "").strip()

In [42]:
import re

# get video title name
def clean_title(text):
    return re.sub(r'[^0-9a-zA-Z\u0E00-\u0E7F\.]', '', text)
title = clean_title(text=title)

# get video id
match = re.search(r"(?:v=|\/)([0-9A-Za-z_-]{11}).*", url)
if match:
    video_id = match.group(1)

In [43]:
from youtube_transcript_api import YouTubeTranscriptApi

# Fetch transcript (auto-captions or uploaded)
transcript = YouTubeTranscriptApi.get_transcript(video_id, languages=['th', 'en'])

# Optionally, save to file
file_name = f'{title}_{video_id}'
with open(f"{file_name}_subtitle.txt", "w", encoding="utf-8") as f:
    for entry in transcript:
        f.write(f"{entry['start']:.2f}s: {entry['text']}\n")
        # f.write(f"{entry['text']}\n")

In [44]:
# import re

# def load_subtitles(file_path):
#     subtitles = []
#     with open(file_path, encoding='utf-8') as f:
#         for line in f:
#             match = re.match(r'([0-9.]+)s:\s(.+)', line.strip())
#             if match:
#                 start_time = float(match.group(1))
#                 text = match.group(2)
#                 subtitles.append({'start': start_time, 'text': text})
#     return subtitles

# def chunk_subtitles(subtitles, chunk_size=60, overlap=20):
#     """
#     Chunk subtitles into segments of `chunk_size` seconds with `overlap` seconds.
#     """
#     chunks = []
#     max_time = subtitles[-1]['start']
#     start_time = 0

#     while start_time <= max_time:
#         end_time = start_time + chunk_size
#         chunk_text = []
#         for entry in subtitles:
#             if start_time <= entry['start'] < end_time:
#                 chunk_text.append(entry['text'])
#         if chunk_text:
#             chunks.append({
#                 'start': start_time,
#                 'end': end_time,
#                 'text': ' '.join(chunk_text)
#             })
#         start_time += chunk_size - overlap
#     return chunks

# subtitles = load_subtitles(f"{file_name}_subtitle.txt")
# chunk_dict = chunk_subtitles(subtitles, chunk_size=600, overlap=60)
# # chunk_dict = chunk_subtitles(subtitles, chunk_size=300, overlap=30)

In [45]:
import re
from pythainlp.tokenize import word_tokenize

def read_text_file(file_path):
    """Read text from a file and return as a string."""
    with open(file_path, 'r', encoding='utf-8') as f:
        return f.read()

def remove_timestamps(text):
    """Remove timestamps like '0.12s:' at the start of each line."""
    cleaned_text = re.sub(r'^\s*\d+\.\d+s:\s*', '', text, flags=re.MULTILINE)
    return cleaned_text

def remove_newlines(text):
    """Remove newline characters."""
    return text.replace('\n', '')

def chunk_thai_text(text, max_tokens, overlap):
    """
    Chunk Thai text using PyThaiNLP word tokenizer with overlap.
    """
    words = word_tokenize(text, keep_whitespace=False)
    chunks = []
    start = 0

    while start < len(words):
        end = min(start + max_tokens, len(words))
        chunk = ''.join(words[start:end])  # Join with no spaces for Thai
        chunks.append(chunk)
        start += max_tokens - overlap

    return chunks

# Example usage:
file_path = f"{file_name}_subtitle.txt"
text = read_text_file(file_path)
text = remove_timestamps(text)
text = remove_newlines(text)
chunks = chunk_thai_text(text, max_tokens=1000, overlap=200)

In [46]:
# from package.model import ModelManager, OllamaRunner
# from package.prompt import Prompt

# system_prompt = """
# Read the following conversation and identify the main topics discussed. 
# Return only a concise list of topics, each as a short phrase or keyword, without extra explanation. 
# Avoid duplicates or overly specific details.
# """


# manager = ModelManager()
# manager.register_runner("llama3", OllamaRunner())

# user_prompt = chunks[0]
# prompt = Prompt(model_name="llama3", user_prompt=user_prompt, system_prompt=system_prompt)

# response = manager.run(prompt)

# print(response)


## Use “Key points extraction” instead of general summary
## Use hierarchical summarization

In [37]:
from package.prompts import KEY_POINTS_PROMPT, SECTION_MERGE_PROMPT, FINAL_SUMMARY_PROMPT
from package.model.ollama import query_ollama
from package.model.gemini import query_gemini

In [48]:
# === PIPELINE ===

chunk_key_points = []
print("Extracting key points for each chunk...")
for i, chunk in enumerate(chunks):
    result = query_ollama(prompt=KEY_POINTS_PROMPT, user_prompt=chunk)
    print(f"Chunk {i+1} done.")
    chunk_key_points.append(result)

# Group into sections, e.g., 3 chunks per section
SECTION_SIZE = 3
section_key_points = []

print("\nMerging chunks into sections...")
for i in range(0, len(chunk_key_points), SECTION_SIZE):
    group = chunk_key_points[i:i+SECTION_SIZE]
    group_text = "\n\n".join(group)
    result = query_ollama(prompt=SECTION_MERGE_PROMPT, user_prompt=group_text)
    print(f"Section {i//SECTION_SIZE + 1} done.")
    section_key_points.append(result)

# Final merge
print("\nMerging all sections into final summary...")
all_sections_text = "\n\n".join(section_key_points)
final_summary = query_ollama(prompt=FINAL_SUMMARY_PROMPT, user_prompt=all_sections_text)
print('Final summary done.')

Extracting key points for each chunk...
Chunk 1 done.
Chunk 2 done.
Chunk 3 done.
Chunk 4 done.
Chunk 5 done.
Chunk 6 done.
Chunk 7 done.

Merging chunks into sections...
Section 1 done.
Section 2 done.
Section 3 done.

Merging all sections into final summary...
Final summary done.


In [49]:
print(final_summary)

Here is a **single, well-organized summary** of the entire video:

**Facts**
• [Fact] The company's revenue in Q2 is 36% higher than last year, with a significant increase in Asia.
• [Fact] Revenue in Q1 was lower than expected, but it has since recovered in Q2.
• [Fact] The company's stock price has dropped by around 10%.
• [Fact] The company's debt-to-equity ratio is around 10,200 million baht.
• [Fact] The company's ROE is around 20%, with a net profit margin of around 17%.

**Ideas**
• [Idea] The speaker believes that the company's poor performance is due to its Q2 results, which were disappointing.
• [Idea] Expanding the product line to include new categories can attract new customers; creating a strong brand identity can attract new customers.
• [Idea] The company should focus on marketing to younger generations, such as those in the Philippines and Indonesia; localize its marketing efforts for each region; build relationships with influencers and content creators in each region.

In [34]:
print(final_summary)

Here is a **single, well-organized summary**:

**Facts**

* [Fact] People with a lot of money may struggle to find genuine friends.
* [Fact] Having good friends is important, but it's not necessary to have many. A few genuine connections can be valuable.
* [Fact] Having a certain amount of money (around 2 million baht) can provide privileges, such as being treated with respect and not needing to worry about small things.

**Ideas**

* [Idea] Having financial knowledge is crucial for achieving wealth and avoiding financial pitfalls.
* [Idea] Having a clear understanding of financial concepts and being able to apply them effectively is key to achieving wealth.
* [Idea] Investing in things one loves can lead to financial success.
* [Idea] Having a few quality connections can be valuable.

**Quotes**

* [Quote] "มันไม่ได้อยู่ที่ปริมาณ มันอยู่ที่คุณภาพ" (It's not about quantity, it's about quality).
* [Quote] "People are equal as long as we don't take money from each other."
* [Quote] "Mone

In [20]:
with open(f"{file_name}_summary.txt", "w", encoding="utf-8") as f:
    f.write(final_summary)